In [ ]:

# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

   
import os
import tensorflow as tf
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold

# You can write up to 20GB to the current directory (/kaggle/wo`rking/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
path = '/kaggle/input/happy-whale-and-dolphin/'
os.listdir(path)

In [ ]:
all_data = pd.read_csv(path+'train.csv')
samp_subm = pd.read_csv(path+'sample_submission.csv')

In [ ]:
train_data = all_data.copy()
train_data.image = train_data.image.apply(lambda x: '/kaggle/input/happy-whale-and-dolphin/train_images/' + x)

train_data.species = train_data.species.apply(lambda x: 'bottlenose_dolphin' if x == 'bottlenose_dolpin' else x)
train_data.species = train_data.species.apply(lambda x: 'killer_whale' if x == 'kiler_whale' else x)

lbl_coder = LabelEncoder()
lbl_coder.fit(train_data.species)


train_data['species_id'] = lbl_coder.transform(train_data.species)
train_data.sample(4)

In [ ]:
folds = list(KFold().split(train_data))
fold_id = 0

csv_train = train_data.iloc[folds[fold_id][0]]
csv_val = train_data.iloc[folds[fold_id][1]]

In [ ]:
set(csv_train.image.apply(lambda x: x.split('.')[-1]))

In [ ]:


def load_img(x):
    img_data = tf.io.read_file(x)
    img = tf.image.decode_jpeg(img_data, channels=3)
    #if img.shape[-1] != 3:
     #   img = tf.image.grayscale_to_rgb(img)
    return img

def normalize(img):
    img = tf.image.resize(img, (128, 128))
    
    img /= 255
    return img

def encode_l(x):
    return tf.one_hot(x, 28)

def load_base_ds(paths, labels):
    ds_i = tf.data.Dataset.from_tensor_slices(paths)
    ds_i = ds_i.map(load_img).map(normalize)
    ds_l = tf.data.Dataset.from_tensor_slices(labels).map(encode_l)
    ds = tf.data.Dataset.zip((ds_i, ds_l))
    return ds
    
ds_train = load_base_ds(csv_train.image, csv_train.species_id)
ds_val = load_base_ds(csv_val.image, csv_val.species_id)


In [ ]:
import tensorflow_hub as hub

m = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2", trainable=False),
    tf.keras.layers.Dense(28, activation='softmax'),
])

m.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(),
         metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [ ]:
def prepare_to_train(ds):
    return ds.shuffle(128).batch(32).prefetch(4)

_ds_train = prepare_to_train(ds_train)
_ds_val = prepare_to_train(ds_val)

m.fit(_ds_train, validation_data=_ds_val, epochs=3)